# Ernesto Antonio Retes Ramírez 

# Procesamiento de Lenguaje Natural 

# Tarea 4: Statistical Language Models

# AYUDA

Mi compañero Daniel me ayudó en el punto 2.1 para la creación del corpus y del vocabulario. Me ayudó de la siguiente manera: yo tenía duda sobre la asignación de $<s>$ y $<s>$, ya que no tenía claro cuantos EOS tenía que agregar. Él me explicó que era necesario agregar un EOS para distinguir entre oraciones, pero que con uno solo bastaba, no era como en el caso de BOS, ya que al estar tomando los dos elementos por detrás de ti en los modelos de trigramas, no necesita agregar más para calcular todas las probabilidades. También me dijo que, desde su punto de vista, podría ser ventajoso agregar dos BOS a cada tweet y ese corpus utilizar para todos los modelos, ya que de cierta forma queremos trabajar sobre el mismo conjunto de datos para poder medirlo con la perplejidad y compararlos.


Por otro lado, ayudé a mi compañero Daniel con respecto al punto 2.3, ya que estaba utilizando la definición exacta de perplejidad la cual da unos ciertos problemas numéricos. Mi ayuda fue mostrarle y explicarle otro forma de clacular la perplejidad utilizando la entropía, que es un método el cual explican en el capitulo de Dan como tema avanzado en el final. 

# 2. Modelo de Lenguaje y Evaluación 

1. Preprocese todos los tuits de agresividad (positivos y negativos) según su intuición para construir un buen corpus para un modelo de lenguaje (e.g., solo palabras en minúscula, etc.). Agregue tokens especiales de $<s>$ y $</s>$ según usted considere (e.g., al inicio y final de cada tuit). Defina su vocabulario y enmascare con <unk> toda palabra que no esté en su vocabulario.

In [157]:
import os
import re
from keras.preprocessing.text import Tokenizer
import numpy as np
import math
from sklearn.model_selection import train_test_split
import wget
import re
from collections import Counter
from time import time
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

In [245]:
def get_texts_from_file(path_corpus,path_truth):
    tr_text = []
    tr_y = []
    
    with open(path_corpus,"r",encoding="utf-8") as f_corpus, open(path_truth,"r",encoding="utf-8") as f_truth:
        for twitt in f_corpus:
            tr_text += [twitt]
        for label in f_truth:
            tr_y += [label]
    return tr_text,tr_y

In [246]:
#Cargamos el conjunto de datos 
tr_txt, tr_y = get_texts_from_file("mex_train.txt","mex_train_labels.txt")
test_txt, test_y = get_texts_from_file("mex_val.txt","mex_val_labels.txt")

In [247]:
tr_txt, val_txt = train_test_split(tr_txt, test_size=616, random_state=42)

In [248]:
import nltk 
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

In [249]:
# hacemos minuscula el texto y agregamos inicio(<s>) y final(</s>) de tweet dependiendo el caso
corpus = []

for doc in tr_txt:
    corpus += ["<s>"] + ["<s>"] + tokenizer.tokenize(doc) + ["</s>"] 
    
fdist1 = nltk.FreqDist(corpus)

In [250]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key],key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [251]:
#Creamos nuestros vocabularios con las 10000 palabras más frecuentes
Vocabulary = sortFreqDict(fdist1)
Vocabulary = Vocabulary[:10000]

In [252]:
dict_indices = dict()
cont = 0 

for weight, word in Vocabulary:
    dict_indices[word] = cont
    cont += 1

In [253]:
# Dado nuestro vocabulario, las palabras que no están en él las cambiamos por "unk"
train = [ w if w in dict_indices else '<unk>' for w in corpus]
N = len(train)

In [254]:
#Vocabulario con unk
fdist2 = nltk.FreqDist(train)
V = sortFreqDict(fdist2)

dict_ind = dict()
cont = 0 

for weight, word in V:
    dict_ind[word] = cont
    cont += 1

El corpus que creamos llamdo train lo hice de la siguiente forma. A cada tweet le agregé dos $<s>$ al inicio y un $</s>$ al final. Esto ya que queremos trabajar hasta con trigramas. Tokenize los tweets y formé mi vocabulario con las 10000 palabras más frecuentes. Finalmente enmascaré las palabras que no pertenecen al corpus con el caracter $<unk>$.

2. Entrene tres modelos de lenguaje sobre todos los tuits: Unigramas(wn1), bigramas(wn1),trigramas(wn1). Para cada uno proporcione una interfaz (función) sencilla para Pn−grama(wn1)y Pn−grama(wn1|wn−1n−N+1). Los modelos deben tener una estrategia común para lidiar consecuencias no vistas. Puede optar por un suavizamiento Laplace o un Good-Turingdiscounting. Muestre un par de ejemplos de como funciona, al menos uno con unapalabra fuera del vocabulario.

UNIGRAMAS

In [255]:
#Función para calcular la probabilidad de una palabra    

P_uni = {}
N = len(train)
len_V = len(V)
for frec, word in V:
    P_uni[word] = (frec+1) / (N+len_V)

In [256]:
#Contiene las frecuencias de cada palabra
freqUni = Counter(train)

Probando algunos ejemplos

In [71]:
P_uni["madre"]

0.008255638314495547

In [17]:
P_uni["<s>"]

0.08556869281386183

In [72]:
P_uni["<unk>"]

0.023048075420594824

In [19]:
P_uni["puto"]

0.0012587461152490582

BIGRAMAS

In [257]:
#Creamos nuestros bigramas
bigrams = [(item[0],item[1]) for item in nltk.bigrams(train)]
freqBigr = Counter(bigrams)

In [258]:
def P_bigr(bigram, freq={}, freqU={}, cond=False):
    if cond:
        if bigram not in freq:
            return 1 / (freqU[bigram[0]]+len_V)
        return (freq[bigram]+1) / (freqU[bigram[0]]+len_V)
    
    else:
        ans = np.log(P_uni[bigram[1]])
        ans += np.log((freq[bigram]+1) / (freqU[bigram[0]]+len_V))
        return  np.exp(ans)

In [76]:
bigram = bigrams[0]
print('P', bigram)
print(P_bigr(bigram, freqBigr, freqUni))
print('\nP(', bigram[1],'|',bigram[0],')')
print(P_bigr(bigram, freqBigr, freqUni, cond=True))

P ('<s>', '<s>')
0.021240272290855868

P( <s> | <s> )
0.2482248073727149


In [77]:
bigram = bigrams[23]
print('P', bigram)
print(P_bigr(bigram, freqBigr, freqUni))
print('\nP(', bigram[1],'|',bigram[0],')')
print(P_bigr(bigram, freqBigr, freqUni, cond=True))

P ('</s>', '<s>')
0.028245864973321128

P( <s> | </s> )
0.3300957867238261


Notemos que de tener un signo de final siga uno de inicio es un hecho muy probable, tomando en cuenta que hemos hecho un
suavizamiento laplaciano donde toda la masa de probabilidad se ha movido. La probabilidad no es cerca a 1 porque hemos movido toda la masa de probabilidad con el suavizamiento laplaciano. Pero aún así es muy alta. 

TRIGRAMAS

In [259]:
trigrams = [(item[0],item[1],item[2]) for item in nltk.trigrams(train)]
freqTrigr = Counter(trigrams)

In [260]:
def P_trigr(trigram, freq={}, freqB={}, freqU={}, cond=False):
    lenVB = len(freqB)
    if cond:
        if trigram not in freq:
            return 1 / (freqB[(trigram[0],trigram[1])] + lenVB)
        return (freq[trigram]+1) / (freqB[(trigram[0],trigram[1])] + lenVB)
    
    else:
        ans = np.log(P_uni[trigram[2]])
        ans += np.log(P_bigr((trigram[1],trigram[2]), freqB, freqU, True))
        ans += np.log((freq[trigram]+1) / (freqB[(trigram[0],trigram[1])] + lenVB))
        return  np.exp(ans)

Veamos un ejemplo

In [84]:
trigram = trigrams[1]
print('P', trigram)
print(P_trigr(trigram, freqTrigr, freqBigr, freqUni))
print('\nP(', trigram[2],'|',trigram[0],trigram[1],')')
print(P_trigr(trigram, freqTrigr, freqBigr, freqUni, cond=True))

P ('<s>', 'vamos', 'a')
2.820764452218357e-09

P( a | <s> vamos )
8.213383708753413e-05


In [87]:
trigram = trigrams[23]
print('P', trigram)
print(P_trigr(trigram, freqTrigr, freqBigr, freqUni))
print('\nP(', trigram[2],'|',trigram[0],trigram[1],')')
print(P_trigr(trigram, freqTrigr, freqBigr, freqUni, cond=True))

P ('</s>', '<s>', '<s>')
0.0019519629615347169

P( <s> | </s> <s> )
0.09189914963449201


Como se esperaba el trigrama de un signo final con dos iniciales es uno que esperaríamos tenga probabilidad alta ya que se repite muchísimo, al nosotros agregarlo a cada tweet. Otro trigrama como el primero es poco común por eso tiene una probabilidad muy baja aunque como podemos ver no es cero. Por lo que en general, los resultados de cada uno de los modelos son consistentes con lo esperado, y podríamos decir son buenos modelos. 

3. Construya un modelo interpolado con valores λ fijos:P̂(wn|wn−2wn−1) = λ1P(wn|wn−2wn−1) + λ2P(wn|wn−1) + λ3P(wn)Para ello experimente con el modelo en particiones estratificadas de 80%, 10% y 10%
para entrenar (train), ajuste de parámetros (val) y prueba (test) respectivamente. Muestrecomo bajan o suben las perplejidades en validación, finalmente pruebe una vez entest. Para esto puede explorar algunos valores ⃗λ y elija el mejor. Pruebe las siguientes:[1/3, 1/3, 1/3],[.4, .4, .2],[.2, .4, .4],[.5, .4, .1] y [.1, .4, .5].

In [261]:
import math

def interpolated_p(trigram, l1, l2, l3):
    return l3*P_trigr(trigram, freqTrigr, freqBigr, freqUni,cond=True) + l2*P_bigr((trigram[1],trigram[2]),freqBigr, freqUni,cond=True) + l1*P_uni[trigram[2]]

def perplexity(corpus, l1, l2, l3,s):
    N = len(corpus) - 2*s
    p = 0
    
    p = sum([np.log(interpolated_p((corpus[i-2],corpus[i-1],corpus[i]),l1,l2,l3)) for i in range(2,len(corpus))])
    H = (-1/N)*p
    return 2**H

In [93]:
#Procesamos el conjunto de validación 
validation = []

for doc in val_txt:
    validation += ["<s>"] + ["<s>"] + tokenizer.tokenize(doc) + ["</s>"] 
    
validation = [ w if w in dict_indices else '<unk>' for w in validation]
s = len(val_txt)

In [96]:
# Tests 1
l1 = 1/3;l2 = 1/3;l3 = 1/3
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.4;l2 = 0.4;l3 =0.2
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.2;l2 =0.4;l3 =0.4
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.5;l2 =0.4;l3 =0.1
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.1;l2=0.4;l3 =0.5
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

for l1=0.3333333333333333 l2=0.3333333333333333 l3=0.3333333333333333 perp=110.55313668294157
for l1=0.4 l2=0.4 l3=0.2 perp=97.7721684986476
for l1=0.2 l2=0.4 l3=0.4 perp=129.78369190056088
for l1=0.5 l2=0.4 l3=0.1 perp=88.37834023431512
for l1=0.1 l2=0.4 l3=0.5 perp=164.34575664442343


In [98]:
# Tests 2
l1 =0.18;l2=0.8;l3=0.02
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.06;l2 = 0.8;l3 =0.14
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.05;l2 =0.85;l3 =0.1
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.7;l2 =0.15;l3 =0.15
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.8;l2=0.1;l3 =0.1
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

for l1=0.18 l2=0.8 l3=0.02 perp=106.26929194702721
for l1=0.06 l2=0.8 l3=0.14 perp=140.89781629360468
for l1=0.05 l2=0.85 l3=0.1 perp=141.9018502613383
for l1=0.7 l2=0.15 l3=0.15 perp=87.02942984781916
for l1=0.8 l2=0.1 l3=0.1 perp=83.68766447504112


In [99]:
# Test 3
l1 = 0.9;l2 =0.05;l3 =0.05
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.8;l2 =0.15;l3 =0.05
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 0.8;l2=0.05;l3 =0.15
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

l1 = 1;l2=0;l3 =0
perp = perplexity(validation, l1, l2, l3, s)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

for l1=0.9 l2=0.05 l3=0.05 perp=81.1056816940672
for l1=0.8 l2=0.15 l3=0.05 perp=80.79218952940077
for l1=0.8 l2=0.05 l3=0.15 perp=87.05767624051929
for l1=1 l2=0 l3=0 perp=79.20123264533848


Vamos a tomar los valores de $\lambda_1 = 0.7, \lambda_2 = 0.15, \lambda_3 = 0.15$, los cuales presentaron un nivel de perplejidad de 87.03 apróximadamente, el cual a comparación de las demás pruebas, es un valor bajo pero sin llegar a ser de los más bajos para evitar el sobreajuste. 

Finalmente probabmos dichos valores una vez con el conjunto test

In [100]:
#Procesamos el conjunto de test
test = []

for doc in test_txt:
    test += ["<s>"] + ["<s>"] + tokenizer.tokenize(doc) + ["</s>"] 
    
test = [ w if w in dict_indices else '<unk>' for w in test]
t = len(test_txt)

In [104]:
l1 = 0.7;l2 =0.15;l3 =0.15
perp = perplexity(test, l1, l2, l3, t)
print("for l1={} l2={} l3={} perp={}".format(l1, l2, l3, perp))

for l1=0.7 l2=0.15 l3=0.15 perp=85.58778125415155


Como podemos ver sobre el conjunto de test también se obtiene una perplejidad muy buena, ya que es bajo a comparación de las demás, por lo que dichos lambda son una buena elección para nuestro modelo. 

# 3. Generación de texto

Para esta parte reentrenará su modelo de lenguaje interpolado para aprender los valores λ:

P̂(wn|wn−2wn−1) = λ1P(wn|wn−2wn−1) + λ2P(wn|wn−1) + λ3P(wn)

Realice las siguientes actividades:

1. Proponga una estrategia con base en Expectation Maximization (investigue porsu cuenta sobre EM) para encontrar buenos valores de interpolación en P̂ usando todoel dataset de agresividad (Se adjunta un material de apoyo). Para ello experimente conel modelo en particiones estratificadas de 80%, 10% y 10% para entrenar (train), ajustarparámetros (val) y probar (test) respectivamente. 1 Muestre como bajan las perplejidadesen 5 iteraciones que usted elija (de todas las que sean necesarias de acuerdo a su EM) envalidación, y pruebe una vez en test. Sino logra hacer este punto, haga los siguientes doscon el modelo de lenguaje con algunos λ fijos.

Para este punto, como nos pide trabajar con el mismo conjunto de tweets vamos a utilizar los conjuntos de entrenamientos, validación y testeo que ya teníamos procesados y para los cuales tenemos toda su información y métricas necesarias. 

In [147]:
def cond_p_ngram(words, n):
    if n>3: 
        n=3
    if n == 1:
        return P_uni[words[-1]]
    if n == 2:
        return P_bigr((words[-2],words[-1]), freqBigr, freqUni,cond=True)
    if n == 3:
        return P_trigr((words[-3],words[-2],words[-1]), freqTrigr, freqBigr, freqUni, cond=True)

In [148]:
def EM_estimate_lambda(words, iter, l1_0, l2_0, l3_0, s):
    # Punto inicial 
    l = np.array([l1_0, l2_0, l3_0])

    M = len(words)   
    
    q = np.zeros(3, dtype =float)
    for i in range(iter):
        # E step
        for m in range(2,M):
            for z in range(3):
                q[z] = cond_p_ngram(words[m-2:m+1], z+1) * l[z]
            q = q/sum(q)
            # Conituous M step
            for z in range(3):
                l[z] += q[z]
        l=l/M
        #print(i)
        #print(l)

        if(i % int(iter/10) ==0 or i==iter-1):  
            perp = perplexity(words, l[0], l[1], l[2], s)
            print(i, l, "    perplexity:", perp)
    
    return l

In [109]:
l = EM_estimate_lambda(validation, 50, 1/3, 1/3, 1/3, s)

0 [8.24321293e-01 1.75480843e-01 1.20096960e-04]     perplexity: 78.1490959001637
5 [8.45640464e-01 1.54281763e-01 1.13940609e-21]     perplexity: 78.08864173968355
10 [8.45640997e-01 1.54281231e-01 1.11284437e-38]     perplexity: 78.08864090712801
15 [8.45640997e-01 1.54281231e-01 1.08690271e-55]     perplexity: 78.0886409071086
20 [8.45640997e-01 1.54281231e-01 1.06156577e-72]     perplexity: 78.0886409071086
25 [8.45640997e-01 1.54281231e-01 1.03681947e-89]     perplexity: 78.0886409071086
30 [8.45640997e-001 1.54281231e-001 1.01265003e-106]     perplexity: 78.0886409071086
35 [8.45640997e-001 1.54281231e-001 9.89044005e-124]     perplexity: 78.0886409071086
40 [8.45640997e-001 1.54281231e-001 9.65988265e-141]     perplexity: 78.0886409071086
45 [8.45640997e-001 1.54281231e-001 9.43469981e-158]     perplexity: 78.0886409071086
49 [8.45640997e-001 1.54281231e-001 2.32558959e-171]     perplexity: 78.0886409071086


Probando con el Test

In [110]:
perp = perplexity(test, l[0], l[1], l[2], t)
print("perplexity on test data:" , perp)

perplexity on test data: 76.78595250067892


Como podemos notar, el algoritmo basado en EM convergió a valores de lambdas para los cuales la perplejidad es más baja que la que habíamos escogido en el problema anterior. Revisando nuevamente los valores de perplejidad obtenidos de manera experimental podemos recordar que los más bajos se encontraban entre 70 y 71, por lo que un valor de 76 sería aún mejor que el de 85 obtenido anteriormente, por ser más bajo, y aún bueno en el sentido de no llegar al sobreajuste. 

2. Haga una función "tuitear" con base en su modelo de lenguaje P̂ del último punto.
El modelo deberá poder parar automáticamente cuando genere el símbolo de terminación
de tuit al final (e.g., "</s>"), o 50 palabras. Proponga algo para que en los últimos tokens
sea más probable generar el token "</s>". Muestre al menos cinco ejemplos.

Función para generar tweets

In [119]:
import random

def generate_next(wk, wj, avoid, l, probfin):
    next = []
    for tg in freqTrigr.keys():
        if tg[:-1]==(wk, wj) and not tg[-1] in avoid:
            p = interpolated_p((wk, wj,tg[-1]), l[0], l[1], l[2])
            p = p*probfin if tg[-1]=='</s>' else p
            next.append((p, tg[-1]))     
    return next
            
    
def tuitear(num_w, l, r, s):
    wj = '<s>'
    wk = '<s>'
    tweet = [wj]
    avoid = ['<s>', '<unk>']
    
    random.seed(s)
    for i in range(num_w):
        next = generate_next(wk, wj, avoid, l, num_w/(i+1))
        if next:
            next = sorted(next, reverse = True)[:r] if not wj=='<s>' else next
            w = next[random.randint(0, len(next)-1)]
            w = w[1]
            tweet.append(w)
            if w == '</s>':
                return tweet
        else:
            next = [(P_bigr((wj,bg[-1]), freqBigr, freqUni,cond=True),bg[-1])\
                    for bg in freqBigr.keys()\
                    if bg[-2]==wj and not bg[-1] in avoid]
            if next:
                next = sorted(next, reverse = True)[:r] if not wj=='<s>' else next
                w = next[random.randint(0, len(next)-1)]
                w = w[1]
                tweet.append(w)
                if w == '</s>':
                    return tweet
        wk = wj
        wj = tweet[-1]
    return tweet

In [123]:
gen_tweet = tuitear(50, l, 5, random.random())
gen_tweet = ' '.join(gen_tweet)
print(gen_tweet)

<s> #pontelaverde .. a ver putos yo ! - no me a dado mas fuerte a ese maricon ... méxico tiene un nombre joto en sus putas se dio cuenta . 😂 </s>


In [124]:
gen_tweet = tuitear(50, l, 5, random.random())
gen_tweet = ' '.join(gen_tweet)
print(gen_tweet)

<s> mujeres : 1 . ¿ a ti que te voy a pinche rodilla y que de la loca que se me para la comunidad lgbt . puras putas pendejadas </s>


In [126]:
gen_tweet = tuitear(50, l, 5, random.random())
gen_tweet = ' '.join(gen_tweet)
print(gen_tweet)

<s> girardi es la de tu madre y que de vez en cuando había visita en la que no ! a quien se la madre . no se me hace ver que se que putas . ! jajaja </s>


In [127]:
gen_tweet = tuitear(50, l, 5, random.random())
gen_tweet = ' '.join(gen_tweet)
print(gen_tweet)

<s> guapuras mañana lunes a partir de las de jenny ya se es la de tu puta y vaciarme en tu vida se resume a estar en la @usuario para soltar a sus putos de todos los que me no he muerto y tengo los putos . no me a bloqueado


In [133]:
gen_tweet = tuitear(50, l, 5, random.random())
gen_tweet = ' '.join(gen_tweet)
print(gen_tweet)

<s> naaa ... cuando ganamos la final como cagan la verga a las que más les gusta . como es la loca y que de de las mujeres no pueden tener hijos </s>


Como podemos notar los tweets generados son bastantes buenos dado que es un modelo n-gramas, algunos cuentan con cierta coherencia como se esperaba, esto es signo de un buen modelo. 

3. Use la intuición que ha ganado en esta tarea y los datos de las mañaneras para
entrenar un modelo de lenguaje AMLO. Haga una un función "dar_conferencia()". Generé
un discurso de 300 palabras y detenga al modelo de forma abrupta.

In [136]:
# Cargamos los datos de las mañaneras que recolectamos en la práctica 1

import glob

str_amlo = ""

for d in glob.glob("estenograficas_limpias_por_fecha/*"):
    with open (d, "r",encoding="utf-8") as file:
        t_conferencia = file.read().replace("\n", " ").lower()
    str_amlo += t_conferencia
    
str_amlo = str_amlo.split()

In [231]:
P_uni = {}
N = len(str_amlo)
freqUni = nltk.FreqDist(str_amlo)
len_V = len(freqUni)
for word in freqUni:
    P_uni[word] = (freqUni[word]+1) / (N+len_V)
    
bigrams = [(item[0],item[1]) for item in nltk.bigrams(str_amlo)]
freqBigr = Counter(bigrams)

trigrams = [(item[0],item[1],item[2]) for item in nltk.trigrams(str_amlo)]
freqTrigr = Counter(trigrams)

Entrenamos nuestro modelo

In [150]:
l = EM_estimate_lambda(str_amlo, 10, 0.3, 0.3, 0.3, 0)

0 [5.33509228e-01 4.66490603e-01 6.98296002e-08]     perplexity: 107.35717488045842
1 [5.33534596e-01 4.66465314e-01 1.60742630e-14]     perplexity: 107.35715565754903
2 [5.33534600e-01 4.66465310e-01 3.70018801e-21]     perplexity: 107.35715565660263
3 [5.33534600e-01 4.66465310e-01 8.51758572e-28]     perplexity: 107.35715565659874
4 [5.33534600e-01 4.66465310e-01 1.96069136e-34]     perplexity: 107.35715565659874
5 [5.33534600e-01 4.66465310e-01 4.51338059e-41]     perplexity: 107.35715565659874
6 [5.33534600e-01 4.66465310e-01 1.03895008e-47]     perplexity: 107.35715565659874
7 [5.3353460e-01 4.6646531e-01 2.3915937e-54]     perplexity: 107.35715565659874
8 [5.33534600e-01 4.66465310e-01 5.50528901e-61]     perplexity: 107.35715565659874
9 [5.33534600e-01 4.66465310e-01 1.26728077e-67]     perplexity: 107.35715565659874


In [151]:
perp = perplexity(str_amlo, l[0],l[1],l[2], 0)
print("perplexity on test data:" , perp)

perplexity on test data: 107.35715565659874


Funciones para generar la conferencia 

In [232]:
def generate_next_amlo(wk, wj, l):
    next = []
    for tg in freqTrigr.keys():
        if tg[:-1]==(wk, wj):
            p = interpolated_p((wk, wj,tg[-1]), l[0], l[1], l[2])
            next.append((p, tg[-1]))     
    return next
            
def gen_amlo(w0, w1, num_w, l, r, s):
    wk = w0
    wj = w1
    txt = [wk , wj]
    
    random.seed(s)
    for i in range(num_w):
        next = generate_next_amlo(wk, wj, l)
        if next:
            next = sorted(next, reverse = True)[:r]
            w = next[random.randint(0, len(next)-1)]
            w = w[1]
            txt.append(w)
        else:
            next = [ (P_bigr((wj,bg[-1]), freqBigr, freqUni,cond=True),bg[-1])\
                    for bg in freqBigr.keys()\
                      if bg[-2]==wj]
            if next:
                next = sorted(next, reverse = True)[:r]
                w = next[random.randint(0, len(next)-1)]
                w = w[1]
                txt.append(w)
        wk = wj
        wj = txt[-1]
    return txt

In [153]:
amlo = gen_amlo('hola', 'ciudadanos', 300, l, 15, random.random())
amlo = ' '.join(amlo)
print(amlo)

hola ciudadanos están conformes con nada. por eso, que en esta su patria y de no están al alza, lo cual es muy difícil con un programa lo estamos llevando la fiscalía, en este que son y los estados no vamos tener este apoyo del presidente lo respeto a un precio del diésel, pero ya un proceso ya sea los cárteles controlan el gas, pero no de la guardia y para una serie de… presidente andrés lópez obrador es trabajar en las acciones al territorio y es en un buen año 2020 al 2022. el secuestro, a pesar desde que está un tema en el estado es donde se en encontró un déficit, tanto en estados para el día y a ver en una también de la que a las autoridades de lo mismo con mis dos papás fueron vacunados, la mayor atención y lo es porque los derechos más allá que iban tan rápido si me está hablando y nos han entregado lo mejor la fiscalía con los no conservadores, demandando hasta en estos que usted las pruebas con un proceso por este plan y de una persona por parte para dar las condiciones que e

El modelo de generación de discursos de amlo se ve que trabaja muy bien aún cuando usamos algo muy básico como los n-gramas. Esto demuestra el poder de un modelo que aunque es sencillo puede llegar a funcionar muy bien. 

4. Calcule el estimado de cada uno sus modelos de lenguaje (el de tuits y el de amlo)
para las frases: "sino gano me voy a la chingada", "ya se va a acabar la corrupción".

In [154]:
str1 = ["si","no","gano","me","voy","a","la","chingada"]
str2 = ["ya","se","va","a","acabar","la","corrupción"]

Modelo de tweets

In [179]:
l_t = [8.45640997e-001,1.54281231e-001,2.32558959e-171]

In [180]:
p = math.prod([interpolated_p((str1[i-2],str1[i-1],str1[i]),l_t[0],l_t[1],l_t[2]) for i in range(2,len(str1))])
print("Probabilidad: ",p)

Probabilidad:  1.9770300041463435e-16


In [188]:
p = math.prod([interpolated_p((str2[i-2],str2[i-1],str2[i]),l_t[0],l_t[1],l_t[2]) for i in range(2,len(str2))])
print("Probabilidad: ",p)

Probabilidad:  3.2885963171193507e-15


Modelo de AMLO

In [ ]:
p = math.prod([interpolated_p((str1[i-2],str1[i-1],str1[i]),l[0],l[1],l[2]) for i in range(2,len(str1))])
print("Probabilidad: ",p)

In [192]:
p = math.prod([interpolated_p((str2[i-2],str2[i-1],str2[i]),l[0],l[1],l[2]) for i in range(2,len(str2))])
print("Probabilidad: ",p)

Probabilidad:  3.7097640894512125e-12


Simplemente utilizamos los modelos generados anteriormente para probabarlos con nuevas frases.

5. Para cada oración del punto anterior, haga todas las permutaciones posibles.
Calcule su probabilidad a cada nueva frase y muestre el top 3 mas probable y el top 3 menos probable (para ambos modelos de lenguaje). Proponga una frase más y haga lo
mismo.

In [229]:
import itertools

str1_all = list(itertools.permutations(str1)) 
str2_all = list(itertools.permutations(str2)) 
str3 = ["le","mandamos","un","saludo","a","la","mamá","del","niño"]

Modelo de tweets

In [262]:
p = []
for st in str1_all:
    p += [(math.prod([interpolated_p((st[i-2],st[i-1],st[i]),l_t[0],l_t[1],l_t[2]) for i in range(2,len(st))]),st)]

top3 = sorted(p, reverse=True)[:3]    
low3 = sorted(p)[:3]

In [263]:
top3

[(3.5059032118271946e-13,
  ('chingada', 'gano', 'si', 'no', 'me', 'voy', 'a', 'la')),
 (3.5058285650169387e-13,
  ('gano', 'chingada', 'si', 'no', 'me', 'voy', 'a', 'la')),
 (3.250196286423799e-13,
  ('gano', 'chingada', 'no', 'me', 'voy', 'a', 'la', 'si'))]

In [264]:
low3

[(9.104359258638601e-18,
  ('la', 'a', 'gano', 'no', 'chingada', 'voy', 'me', 'si')),
 (9.105177272879079e-18,
  ('la', 'a', 'gano', 'voy', 'no', 'chingada', 'me', 'si')),
 (9.116838762985829e-18,
  ('la', 'a', 'gano', 'me', 'no', 'chingada', 'voy', 'si'))]

In [265]:
p = []
for st in str2_all:
    p += [(math.prod([interpolated_p((st[i-2],st[i-1],st[i]),l_t[0],l_t[1],l_t[2]) for i in range(2,len(st))]),st)]

top3 = sorted(p, reverse=True)[:3]    
low3 = sorted(p)[:3]

In [266]:
top3

[(1.233291494329925e-11,
  ('acabar', 'corrupción', 'ya', 'se', 'va', 'a', 'la')),
 (1.2332902222389657e-11,
  ('corrupción', 'acabar', 'ya', 'se', 'va', 'a', 'la')),
 (1.160360530234272e-11,
  ('acabar', 'corrupción', 'a', 'la', 'ya', 'se', 'va'))]

In [269]:
low3

[(1.6284439153839253e-16,
  ('la', 'a', 'corrupción', 'va', 'se', 'acabar', 'ya')),
 (1.6324001928727816e-16,
  ('la', 'a', 'corrupción', 'se', 'acabar', 'va', 'ya')),
 (1.6337257321296207e-16,
  ('la', 'a', 'corrupción', 'va', 'se', 'ya', 'acabar'))]

In [230]:
#Nueva oración dada por mi
p = math.prod([interpolated_p((str3[i-2],str3[i-1],str3[i]),l_t[0],l_t[1],l_t[2]) for i in range(2,len(str1))])
print("Probabilidad: ",p)

Probabilidad:  1.3132416497093299e-16


Modelo de AMLO

In [241]:
p = []
for st in str2_all:
    p += [(math.prod([interpolated_p((st[i-2],st[i-1],st[i]),l[0],l[1],l[2]) for i in range(2,len(st))]),st)]

top3 = sorted(p, reverse=True)[:3]
low3 = sorted(p)[:3]

In [242]:
top3

[(2.4757653325433096e-09,
  ('acabar', 'corrupción', 'ya', 'se', 'va', 'a', 'la')),
 (2.4605909572043068e-09,
  ('corrupción', 'acabar', 'ya', 'se', 'va', 'a', 'la')),
 (1.6789815745280819e-09,
  ('acabar', 'ya', 'se', 'va', 'a', 'la', 'corrupción'))]

In [243]:
low3

[(2.2435644955463123e-16,
  ('a', 'la', 'acabar', 'va', 'se', 'corrupción', 'ya')),
 (2.2516676044018094e-16,
  ('a', 'la', 'acabar', 'va', 'se', 'ya', 'corrupción')),
 (2.267159067556165e-16,
  ('a', 'la', 'acabar', 'ya', 'corrupción', 'va', 'se'))]

In [ ]:
p = []
for st in str1_all:
    p += [(math.prod([interpolated_p((st[i-2],st[i-1],st[i]),l[0],l[1],l[2]) for i in range(2,len(st))]),st)]

top3 = sorted(p, reverse=True)[:3]   
low3 = sorted(p)[:3]

In [244]:
#Nueva oración dada por mi
p = math.prod([interpolated_p((str3[i-2],str3[i-1],str3[i]),l[0],l[1],l[2]) for i in range(2,len(str1))])
print(str3)
print("Probabilidad: ",p)

['le', 'mandamos', 'un', 'saludo', 'a', 'la', 'mamá', 'del', 'niño']
Probabilidad:  3.3470719702831416e-16


Este punto también es para evaluar los modelos mediante las permutaciones de las oraciones y la tarea es determinar cual es una oración más probable de ser correcta. Los resultados de las top3 fueron buenos, ya que son oraciones con sentido, mientras que las 3 menores fueron lo esperado, oraciones que no tienen sentido. 

# (Opcional) El ahorcado (10pts) acumulables solo para tareas: e.g.; esta tarea puede llegar a valer 110/100

Para esta parte estudie y comprenda el funcionamiento de la estrategia propuesta por
Norvig http://norvig.com/spell-correct.html. Siéntete libre de adaptar y/o extender parcial o
totalmente el código de Norvig para esta tarea.
Diseñe una función que sea capaz de encontrar los caracteres faltantes de una palabra. Para
ello proponga una adaptación simple de la estrategia de corrección ortográfica propuesta por
Norvig. La función de el ahorcado debe poder tratar con hasta 4 caracteres desconocidos en
palabras de longitud arbitraria. La función debe trabajar en tiempo razonable (≈ 1 minuto en
una laptop o menos). La función debe trabajar como sigue con 10 ejemplos:

hangman ( " pe_p_e " )
’ people ’

hangman ( " phi__sop_y " )
’ philosophy ’

hangman ( " s i _ n i f _ c _ n c _ " )
’ s i g n i f i c a n c e ’

Puede resolver este punto con una extensión muy simple de la estrategia de Norvig, o alguna
forma más eficiente con distancias de edición (e.g., Levenshtein) o de subcadenas (e.g., Karp
Rabin, Aho-Corasick, Tries, etc.).

Importamos el texto

In [50]:
url_pag = 'http://norvig.com/big.txt'
pth = ''
wget.download(url = url_pag, out = pth)

'big (4).txt'

In [119]:
"Función del ahorcado"

def words(text): 
    #return re.findall(r'\w+', text.lower())
    return re.findall('[^\d\W]+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def hangman(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or 
            known(ahorcado_edit1(word)) or 
            known(ahorcado_edit2(word)) or 
            known(ahorcado_edit3(word)) or 
            known(ahorcado_edit4(word)) or 
            [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def ahorcado_edit1(word):
    "Función que trata de adivinar una palabras con hasta 4 letras faltantes"
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word)) if word[i]=='_']
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    return set(replaces)

def ahorcado_edit2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in ahorcado_edit1(word) for e2 in ahorcado_edit1(e1))

def ahorcado_edit3(word): 
    "All edits that are 3 edits away from `word`."
    return (e2 for e1 in ahorcado_edit2(word) for e2 in ahorcado_edit1(e1))

def ahorcado_edit4(word): 
    "All edits that are 4 edits away from `word`."
    return (e2 for e1 in ahorcado_edit3(word) for e2 in ahorcado_edit1(e1))

La idea de la función es realizar los splits de las palabras, solamente si se encuentra un espacio vacío. Con dicho espacio solamente se realizará la operación de reemplazo por otra letra. Con esta misma estrategia se buscarán posibles palabras reemplazando hasta 4 veces los espacios vacíos, (fácilmente extendible hasta más letras). Para palabras con más de 1 letra faltante, lo que se hace es realizar los posibles reemplazos con n-1 letras faltantes y luego hacer otro reemplazo extra.


In [122]:
start = time()
word = hangman('sp_l_i__')
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 4.73


'spelling'

In [123]:
start = time()
word = hangman ( "pe_p_e" )
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 0.00


'people'

In [124]:
start = time()
word = hangman ("phi__sop_y")
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 0.04


'philosophy'

In [125]:
start = time()
word = hangman ("si_nif_c_nc_")
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 4.28


'significance'

In [126]:
start = time()
word = hangman ("he__o")
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 0.00


'hello'

In [127]:
start = time()
word = hangman ("refr_g_r_t_r")
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 4.48


'refrigerator'

In [128]:
start = time()
word = hangman ("e_se__ia_")
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 4.27


'essential'

In [129]:
start = time()
word = hangman ("l__gu_ge_")
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 4.42


'languages'

In [130]:
start = time()
word = hangman ("d__ma__")
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 4.17


'denmark'

In [131]:
start = time()
word = hangman ("eu____")
end = time()
print("Tiempo en encontrar la solución: %.2f" % (end-start))
word

Tiempo en encontrar la solución: 4.07


'europe'

Comente brevemente como integraría un modelo de lenguaje con el modelo de Norvig
para tratar de resolver errores gramaticales de más alto nivel, o errores dónde el error sea una
palabra que si está en el diccionario, por ejemplo: "In the science off Maths ...".

Para abordar este problema, se podría utilizar el modelo de lenguaje para generar una lista de posibles correcciones para la oración con errores. A continuación, se podría utilizar el modelo de Norvig para determinar la probabilidad de cada corrección y elegir la más probable.

Una forma de hacer esto sería:

Utilizar el modelo de lenguaje para generar una lista de posibles correcciones para la oración con errores. Esto se puede hacer utilizando técnicas de corrección de errores basadas en modelos de lenguaje, como la corrección de errores basada en estadísticas o la corrección de errores basada en redes neuronales.

Utilizar el modelo de Norvig para determinar la probabilidad de cada corrección. El modelo de Norvig utiliza técnicas de procesamiento de lenguaje natural para determinar la probabilidad de una palabra dada en un contexto determinado. En este caso, se podría utilizar el modelo de Norvig para determinar la probabilidad de cada posible corrección en el contexto de la oración con errores.

Elegir la corrección más probable. Una vez que se han determinado las probabilidades de cada posible corrección utilizando el modelo de Norvig, se podría elegir la corrección más probable como la corrección sugerida para la oración con errores.

En el ejemplo dado, "In the science off Maths ..." se podría identificar que "off" es una palabra incorrecta y proponer como posible corrección la palabra "of". A continuación, el modelo de Norvig podría determinar la probabilidad de que "of" sea la palabra correcta en el contexto de la oración, teniendo en cuenta la estructura gramatical y semántica de la oración. Si "of" tiene una probabilidad alta, entonces se podría elegir como la corrección sugerida.